<a href="https://colab.research.google.com/github/biaemcasa/miranda/blob/main/Miranda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# %%
# Instalação de bibliotecas
# Use uma versão mais recente de flask-ngrok e instale google-adk aqui
!pip -q install google-genai google-adk Flask flask-ngrok

# Configurações de API e cliente Gemini/ADK
import os
from google.colab import userdata
# Certifique-se de ter 'GOOGLE_API_KEY' e 'PEXELS_API_KEY' configuradas nos Secrets do Colab
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["PEXELS_API_KEY"] = userdata.get('PEXELS_API_KEY')

import os
from google import genai
from google.colab import userdata
google_api_key = userdata.get('GOOGLE_API_KEY')
if not google_api_key:
    google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    raise ValueError("GOOGLE_API_KEY não encontrada no ambiente. Verifique se ela está configurada corretamente nos Secrets do Colab ou como variável de ambiente.")
client = genai.Client(api_key=google_api_key)
MODEL_ID = "gemini-2.0-flash"
print("Cliente Gemini configurado com sucesso!")

# Importações adicionais
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
# %%
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [7]:
# %%
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje, links_produtos=None):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca informações no Google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta do google (google_search)
      para recueprar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
      Se forem fornecidos links de produtos, use esses links como ponto de partida para entender
      os produtos específicos do lojista e buscar notícias e informações relevantes sobre eles ou a categoria a que pertencem.
      Foque em no máximo 5 lançamentos relevantes...
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  if links_produtos:
      entrada_do_agente_buscador += f"\nLinks de produtos de referência: {', '.join(links_produtos)}"

  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

# %%
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de conteúdos, especialista em criar posts para blogs de ecommerce.
        Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um texto de blog sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem
        abordados no texto para blog que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

# %%
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para blogs de moda.
            Você escreve posts para blogs de moda ligados a ecommerces. Utilize o tema fornecido no plano de post e os pontos mais relevantes
            fornecidos e, com base nisso, escreva um rascunho de post para blog sobre o tema indicado.
            O tamanho ideal para um texto de blog otimizado geralmente gira em torno de 1.500 a 2.000 palavras.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 tags no final.
            """,
        description="Agente redator de posts engajadores para blogs de moda"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

# %%
############################################
# --- Agente 5: Buscador de Imagens --- #
############################################
def agente_buscador_imagens(topico, rascunho_gerado):
    buscador_imagens = Agent(
        name="agente_buscador_imagens",
        model="gemini-2.0-flash",
        instruction="""
            Você é um buscador de imagens para posts de blog de moda.
            Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar imagens de alta qualidade
            e relevantes para ilustrar um post de blog sobre o tópico e rascunho fornecidos.
            **Priorize a busca de imagens no site Pexels (pexels.com)**, utilizando termos de busca relevantes para o tópico e o conteúdo do post. Se não encontrar imagens adequadas no Pexels, você pode expandir a busca para outros sites de moda confiáveis.
            Para cada imagem encontrada (especialmente do Pexels), forneça a URL da imagem e a URL da página onde ela foi encontrada.
            Liste no máximo 5 imagens relevantes.
            """,
        description="Agente que busca imagens para posts de blog de moda, priorizando o Pexels.",
        tools=[google_search]
    )
    entrada_do_agente_buscador_imagens = f"Tópico: {topico}\nRascunho do post: {rascunho_gerado}"
    imagens_encontradas = call_agent(buscador_imagens, entrada_do_agente_buscador_imagens)
    return imagens_encontradas

# %%
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts virais para blogs de moda, orientados ao SEO do Google, com foco em blogs de moda.
            Por ter um público jovem, entre 21 a 35 anos, use um tom de escrita adequado.
            Revise o rascunho de post blogs de moda, orientados ao SEO do Google abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Além disso, avalie o post com foco em SEO para o Google, considerando os seguintes aspectos:
            - **Uso de Palavras-chave:** O rascunho utiliza palavras-chave relevantes para o tópico de forma natural no título, subtítulos e ao longo do texto? Sugira palavras-chave adicionais relevantes se necessário.
            - **Estrutura do Conteúdo:** O texto está bem estruturado com títulos (H1, H2, H3) e parágrafos curtos para facilitar a leitura?
            - **Links Internos e Externos:** O rascunho inclui oportunidades para links internos (para outros produtos ou posts do blog do ecommerce) e links externos (para fontes relevantes e confiáveis)? Sugira onde links podem ser adicionados.
            - **Legibilidade:** O texto é fácil de ler para o público-alvo (linguagem simples, frases curtas)?
            - **Otimização de Imagens (Alt Text):** Se as imagens fossem incluídas, o rascunho fornece contexto suficiente para gerar bons "alt text" (descrição da imagem) com palavras-chave relevantes? (O Agente 6 cuidará da inclusão, mas o Agente 4 pode avaliar a base no rascunho).
            - **Intenção de Busca:** O conteúdo do post alinha-se com a provável intenção de busca do usuário ao procurar sobre o tópico?

            Se o rascunho estiver ótimo e otimizado para SEO, responda apenas 'O rascunho está ótimo, otimizado para SEO e pronto para publicar!'.
            Caso haja problemas de qualidade ou otimização para SEO, aponte-os de forma clara e sugira melhorias específicas para tornar o post mais forte para SEO.
            """,
        description="Agente revisor de post para blogs de moda, orientados ao SEO do Google."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

# %%
######################################################
# --- Agente 6: Formatador de Post com Imagens --- #
######################################################
def agente_formatador_imagens(rascunho_gerado, imagens_encontradas):
    formatador = Agent(
        name="agente_formatador_imagens",
        model="gemini-2.0-flash",
        instruction="""
            Você é um formatador de posts de blog, especializado em moda.
            Sua tarefa é pegar o rascunho do post e a lista de imagens encontradas e gerar o texto final do post,
            incluindo o código de incorporação das imagens nos locais apropriados dentro do texto.

            Formato de saída: Markdown. Use o seguinte formato para incorporar imagens:
            ![Descrição da Imagem](URL_DA_IMAGEM)

            Analise o texto do rascunho para identificar seções onde uma imagem seria relevante para ilustrar o conteúdo.
            Para cada imagem na lista fornecida, tente encontrar um ponto relevante no texto para inseri-la.
            Use o tópico da imagem ou o contexto do texto para gerar uma breve "Descrição da Imagem" (alt text).
            Não insira todas as imagens se elas não parecerem relevantes para o texto. Foque naquelas que melhor complementam o conteúdo.
            Posicione as imagens de forma que quebrem o texto de forma natural, talvez após um parágrafo relevante.
            Inclua as tags originais do rascunho no final do post formatado.
            """,
        description="Agente que formata o post do blog, incluindo código de incorporação de imagens em Markdown."
    )
    entrada_do_agente_formatador = f"Rascunho do Post:\n{rascunho_gerado}\n\nImagens Encontradas:\n{imagens_encontradas}"
    post_formatado = call_agent(formatador, entrada_do_agente_formatador)
    return post_formatado

In [ ]:
# %%
# --- Configurar o endpoint Flask ---
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok # Importe run_with_ngrok
import json
from datetime import date # Certifique-se que date está importado

app = Flask(__name__)
run_with_ngrok(app) # Use run_with_ngrok para expor a aplicação

@app.route('/', methods=['POST'])
def create_fashion_blog_post():
    data = request.json

    # Extrair dados da requisição
    topico = data.get('topico')
    links_produtos = data.get('links_produtos', [])

    if not topico:
        return jsonify({"error": "Tópico é obrigatório"}), 400

    try:
        # Seu código existente aqui
        data_de_hoje = date.today().strftime("%d/%m/%Y")

        # Executar agentes sequencialmente
        lancamentos_buscados = agente_buscador(topico, data_de_hoje, links_produtos=links_produtos)
        plano_de_post = agente_planejador(topico, lancamentos_buscados)
        rascunho_de_post = agente_redator(topico, plano_de_post)
        post_revisado = agente_revisor(topico, rascunho_de_post)
        imagens_para_post = agente_buscador_imagens(topico, rascunho_de_post)
        post_final_formatado = agente_formatador_imagens(post_revisado, imagens_para_post)

        # Retornar resultado estruturado
        return jsonify({
            "status": "completed",
            "topico": topico,
            "data_criacao": data_de_hoje,
            "resultados": {
                "lancamentos_buscados": lancamentos_buscados,
                "plano_de_post": plano_de_post,
                "rascunho_de_post": rascunho_de_post,
                "post_revisado": post_revisado,
                "imagens_para_post": imagens_para_post,
                "post_final_formatado": post_final_formatado
            }
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Para streaming (opcional) - Descomente e implemente se necessário
# @app.route('/stream', methods=['POST'])
# def create_fashion_blog_post_stream():
#     # Implementar streaming se necessário
#     pass

if __name__ == '__main__':
    # Não precisamos mais do input e da execução direta dos agentes aqui
    # A lógica agora está na rota Flask
    app.run() # Isso iniciará o servidor Flask e ngrok

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura a API Key do Pexels a partir do Colab Secrets
import os
from google.colab import userdata

# Substitua 'CHAVE_PEXELS' pelo nome exato que você usou para a chave do Pexels no Colab Secrets
os.environ["PEXELS_API_KEY"] = userdata.get('PEXELS_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

import os
from google import genai
from google.colab import userdata # Certifique-se de importar userdata aqui também

# Recupera a API key do ambiente. Tenta obter do Colab Secrets primeiro.
google_api_key = userdata.get('GOOGLE_API_KEY')

# Se a chave não foi encontrada nos Secrets, verifica se está em variáveis de ambiente (menos comum em Colab)
if not google_api_key:
    google_api_key = os.getenv("GOOGLE_API_KEY")

# Verifica se a API key foi encontrada
if not google_api_key:
    raise ValueError("GOOGLE_API_KEY não encontrada no ambiente. Verifique se ela está configurada corretamente nos Secrets do Colab ou como variável de ambiente.")

# Passa a API key explicitamente para o construtor do cliente
client = genai.Client(api_key=google_api_key)

MODEL_ID = "gemini-2.0-flash"

print("Cliente Gemini configurado com sucesso!")

Cliente Gemini configurado com sucesso!


In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje, links_produtos=None):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca informações no Google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta do google (google_search)
      para recueprar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
      Se forem fornecidos links de produtos, use esses links como ponto de partida para entender
      os produtos específicos do lojista e buscar notícias e informações relevantes sobre eles ou a categoria a que pertencem.
      Foque em no máximo 5 lançamentos relevantes...
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  if links_produtos:
      entrada_do_agente_buscador += f"\nLinks de produtos de referência: {', '.join(links_produtos)}"

  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdos, especialista em criar posts para blogs de ecommerce.
        Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um texto de blog sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem
        abordados no texto para blog que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para blogs de moda.
            Você escreve posts para blogs de moda ligados a ecommerces. Utilize o tema fornecido no plano de post e os pontos mais relevantes
            fornecidos e, com base nisso, escreva um rascunho de post para blog sobre o tema indicado.
            O tamanho ideal para um texto de blog otimizado geralmente gira em torno de 1.500 a 2.000 palavras.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 tags no final.
            """,
        description="Agente redator de posts engajadores para blogs de moda"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
############################################
# --- Agente 5: Buscador de Imagens --- #
############################################
def agente_buscador_imagens(topico, rascunho_gerado):
    buscador_imagens = Agent(
        name="agente_buscador_imagens",
        model="gemini-2.0-flash",
        instruction="""
            Você é um buscador de imagens para posts de blog de moda.
            Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar imagens de alta qualidade
            e relevantes para ilustrar um post de blog sobre o tópico e rascunho fornecidos.
            **Priorize a busca de imagens no site Pexels (pexels.com)**, utilizando termos de busca relevantes para o tópico e o conteúdo do post. Se não encontrar imagens adequadas no Pexels, você pode expandir a busca para outros sites de moda confiáveis.
            Para cada imagem encontrada (especialmente do Pexels), forneça a URL da imagem e a URL da página onde ela foi encontrada.
            Liste no máximo 5 imagens relevantes.
            """,
        description="Agente que busca imagens para posts de blog de moda, priorizando o Pexels.",
        tools=[google_search] # Continuamos usando a ferramenta de busca do Google
    )
    entrada_do_agente_buscador_imagens = f"Tópico: {topico}\nRascunho do post: {rascunho_gerado}"
    # Executa o agente
    imagens_encontradas = call_agent(buscador_imagens, entrada_do_agente_buscador_imagens)
    return imagens_encontradas

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts virais para blogs de moda, orientados ao SEO do Google, com foco em blogs de moda.
            Por ter um público jovem, entre 21 a 35 anos, use um tom de escrita adequado.
            Revise o rascunho de post blogs de moda, orientados ao SEO do Google abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Além disso, avalie o post com foco em SEO para o Google, considerando os seguintes aspectos:
            - **Uso de Palavras-chave:** O rascunho utiliza palavras-chave relevantes para o tópico de forma natural no título, subtítulos e ao longo do texto? Sugira palavras-chave adicionais relevantes se necessário.
            - **Estrutura do Conteúdo:** O texto está bem estruturado com títulos (H1, H2, H3) e parágrafos curtos para facilitar a leitura?
            - **Links Internos e Externos:** O rascunho inclui oportunidades para links internos (para outros produtos ou posts do blog do ecommerce) e links externos (para fontes relevantes e confiáveis)? Sugira onde links podem ser adicionados.
            - **Legibilidade:** O texto é fácil de ler para o público-alvo (linguagem simples, frases curtas)?
            - **Otimização de Imagens (Alt Text):** Se as imagens fossem incluídas, o rascunho fornece contexto suficiente para gerar bons "alt text" (descrição da imagem) com palavras-chave relevantes? (O Agente 6 cuidará da inclusão, mas o Agente 4 pode avaliar a base no rascunho).
            - **Intenção de Busca:** O conteúdo do post alinha-se com a provável intenção de busca do usuário ao procurar sobre o tópico?

            Se o rascunho estiver ótimo e otimizado para SEO, responda apenas 'O rascunho está ótimo, otimizado para SEO e pronto para publicar!'.
            Caso haja problemas de qualidade ou otimização para SEO, aponte-os de forma clara e sugira melhorias específicas para tornar o post mais forte para SEO.
            """,
        description="Agente revisor de post para blogs de moda, orientados ao SEO do Google."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
######################################################
# --- Agente 6: Formatador de Post com Imagens --- #
######################################################
def agente_formatador_imagens(rascunho_gerado, imagens_encontradas):
    formatador = Agent(
        name="agente_formatador_imagens",
        model="gemini-2.0-flash",
        instruction="""
            Você é um formatador de posts de blog, especializado em moda.
            Sua tarefa é pegar o rascunho do post e a lista de imagens encontradas e gerar o texto final do post,
            incluindo o código de incorporação das imagens nos locais apropriados dentro do texto.

            Formato de saída: Markdown. Use o seguinte formato para incorporar imagens:
            ![Descrição da Imagem](URL_DA_IMAGEM)

            Analise o texto do rascunho para identificar seções onde uma imagem seria relevante para ilustrar o conteúdo.
            Para cada imagem na lista fornecida, tente encontrar um ponto relevante no texto para inseri-la.
            Use o tópico da imagem ou o contexto do texto para gerar uma breve "Descrição da Imagem" (alt text).
            Não insira todas as imagens se elas não parecerem relevantes para o texto. Foque naquelas que melhor complementam o conteúdo.
            Posicione as imagens de forma que quebrem o texto de forma natural, talvez após um parágrafo relevante.
            Inclua as tags originais do rascunho no final do post formatado.
            """,
        description="Agente que formata o post do blog, incluindo código de incorporação de imagens em Markdown."
    )
    entrada_do_agente_formatador = f"Rascunho do Post:\n{rascunho_gerado}\n\nImagens Encontradas:\n{imagens_encontradas}"
    # Executa o agente
    post_formatado = call_agent(formatador, entrada_do_agente_formatador)
    return post_formatado

In [ ]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

print(" Iniciando o Sistema de Criação de Posts para blogs de moda, orientados ao SEO do Google ")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o próximo post do blog: ")

# --- Obter os Links dos Produtos do Lojista (Novo) ---
links_input = input("🔗 Opcional: Digite os links dos produtos do lojista (separados por vírgula): ")
links_do_lojista = [link.strip() for link in links_input.split(',') if link.strip()] # Processa a string de links

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! Vamos então criar o post sobre {topico}")

  # A chamada para o agente_buscador agora usará a lista links_do_lojista
  lancamentos_buscados = agente_buscador(topico, data_de_hoje, links_produtos=links_do_lojista)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("-----------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("-----------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))

  post_revisado = agente_revisor(topico, rascunho_de_post) # Renomeei para post_revisado para clareza
  print("\n--- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_revisado))
  print("-----------------------------------------------------")

  imagens_para_post = agente_buscador_imagens(topico, rascunho_de_post)
  print("\n--- Resultado do Agente 5 (Buscador de Imagens) ---\n")
  display(to_markdown(imagens_para_post))
  print("-----------------------------------------------------")

  # Chamar o novo Agente Formatador de Post com Imagens
  post_final_formatado = agente_formatador_imagens(post_revisado, imagens_para_post)
  print("\n--- Resultado do Agente 6 (Formatador com Imagens) ---\n")
  display(to_markdown(post_final_formatado))
  print("-----------------------------------------------------")

 Iniciando o Sistema de Criação de Posts para blogs de moda, orientados ao SEO do Google 
